# Importing Libraries and Configuration


In [ ]:
import pandas as pd
import numpy as np
import time
import torch
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    pipeline
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import kagglehub
import os
import warnings
warnings.filterwarnings("ignore")

# Load Dataset from KaggleHub

In [ ]:
path = kagglehub.dataset_download("lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")
csv_path = os.path.join(path, "IMDB Dataset.csv")
data = pd.read_csv(csv_path)

In [ ]:
display(data.head())

In [ ]:
display(data.tail())

In [ ]:
data.shape

In [ ]:
type(data)

In [ ]:
data["sentiment"].value_counts()

# Label Encoding for Sentiment Classes

In [ ]:
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)

In [ ]:
data.head()

In [ ]:
data.tail()

# Prepare Data

In [ ]:
train_texts, test_texts, train_labels, test_labels = train_test_split(
    data["review"].tolist(), data["sentiment"].tolist(), test_size=0.2, random_state=42
)

In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_texts, train_labels, test_size=0.2, random_state=42
)

# Data Preprocessing (Tokenization)

In [ ]:
# Common preprocessing function
def preprocess_data(tokenizer):
    def tokenize_function(examples):
        return tokenizer(examples["text"], truncation=True, padding=True, max_length=512)

    train_dataset = Dataset.from_dict({"text": train_texts, "label": train_labels})
    val_dataset = Dataset.from_dict({"text": val_texts, "label": val_labels})
    test_dataset = Dataset.from_dict({"text": test_texts, "label": test_labels})

    tokenized_train = train_dataset.map(tokenize_function, batched=True)
    tokenized_val = val_dataset.map(tokenize_function, batched=True)
    tokenized_test = test_dataset.map(tokenize_function, batched=True)

    return tokenized_train,tokenized_val, tokenized_test

# Metrics Computation Function

In [ ]:
# Shared compute metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

# Train Each Model Separately and Collect Results
### Model 1: nikolasmoya/imdb-binary-sentiment-analysis

In [ ]:
results = list()

# Train imdb model
start_imdb = time.time()
nikolasmoya_model_name = "nikolasmoya/imdb-binary-sentiment-analysis"
nikolasmoya_tokenizer = AutoTokenizer.from_pretrained(nikolasmoya_model_name)
nikolasmoya_model = AutoModelForSequenceClassification.from_pretrained(nikolasmoya_model_name)

nikolasmoya_tokenized_train, nikolasmoya_tokenized_eval, nikolasmoya_tokenized_test = preprocess_data(nikolasmoya_tokenizer)
nikolasmoya_data_collator = DataCollatorWithPadding(tokenizer=nikolasmoya_tokenizer)

nikolasmoya_training_args = TrainingArguments(
output_dir="./results/imdb",
learning_rate=2e-5,
per_device_train_batch_size=16,
per_device_eval_batch_size=16,
num_train_epochs=2,
weight_decay=0.01,
logging_dir='./logs'
)

nikolasmoya_trainer = Trainer(
    model=nikolasmoya_model,
    args=nikolasmoya_training_args,
    train_dataset=nikolasmoya_tokenized_train,
    eval_dataset=nikolasmoya_tokenized_eval,
    tokenizer=nikolasmoya_tokenizer,
    data_collator=nikolasmoya_data_collator,
    compute_metrics=compute_metrics
)

nikolasmoya_trainer.train()
nikolasmoya_results = nikolasmoya_trainer.evaluate()
print("IMDB Model Results:")
print(nikolasmoya_results)
nikolasmoya_preds = np.argmax(nikolasmoya_trainer.predict(nikolasmoya_tokenized_test).predictions, axis=-1)
print(classification_report(test_labels, nikolasmoya_preds, target_names=["Negative", "Positive"]))
end_imdb = time.time()
nikolasmoya_results["model"] = "nikolasmoya/imdb-binary-sentiment-analysis"
nikolasmoya_results["time_sec"] = end_imdb - start_imdb
results.append(nikolasmoya_results)

### Model 2: kubi565/roberta-binary-sentiment-multilingual

In [ ]:
# Train roberta model
start_roberta = time.time()
roberta_model_name = "kubi565/roberta-binary-sentiment-multilingual"
roberta_tokenizer = AutoTokenizer.from_pretrained(roberta_model_name)
roberta_model = AutoModelForSequenceClassification.from_pretrained(roberta_model_name)

roberta_tokenized_train, roberta_tokenized_eval ,roberta_tokenized_test = preprocess_data(roberta_tokenizer)
roberta_data_collator = DataCollatorWithPadding(tokenizer=roberta_tokenizer)

roberta_training_args = TrainingArguments(
output_dir="./results/roberta",
learning_rate=2e-5,
per_device_train_batch_size=16,
per_device_eval_batch_size=16,
num_train_epochs=2,
weight_decay=0.01,
logging_dir='./logs'
)

roberta_trainer = Trainer(
    model=roberta_model,
    args=roberta_training_args,
    train_dataset=roberta_tokenized_train,
    eval_dataset=roberta_tokenized_eval,
    tokenizer=roberta_tokenizer,
    data_collator=roberta_data_collator,
    compute_metrics=compute_metrics
)

roberta_trainer.train()
roberta_results = roberta_trainer.evaluate()
print("roberta Results:")
print(roberta_results)
roberta_preds = np.argmax(roberta_trainer.predict(roberta_tokenized_test).predictions, axis=-1)
print(classification_report(test_labels, roberta_preds, target_names=["Negative", "Positive"]))
end_roberta = time.time()
roberta_results["model"] = "roberta-binary-sentiment-multilingual"
roberta_results["time_sec"] = end_roberta - start_roberta
results.append(roberta_results)

### Model 3: autoevaluate/binary-classification

In [ ]:
# Train binary_classification model
start_binary_classification = time.time()
binary_classification_model_name = "autoevaluate/binary-classification"
binary_classification_tokenizer = AutoTokenizer.from_pretrained(binary_classification_model_name)
binary_classification_model = AutoModelForSequenceClassification.from_pretrained(binary_classification_model_name)

binary_classification_tokenized_train,binary_classification_tokenized_eval, binary_classification_tokenized_test = preprocess_data(binary_classification_tokenizer)
binary_classification_data_collator = DataCollatorWithPadding(tokenizer=binary_classification_tokenizer)

binary_classification_training_args = TrainingArguments(
output_dir="./results/binary-classification",
learning_rate=2e-5,
per_device_train_batch_size=16,
per_device_eval_batch_size=16,
num_train_epochs=2,
weight_decay=0.01,
logging_dir='./logs'
)

binary_classification_trainer = Trainer(
    model=binary_classification_model,
    args=binary_classification_training_args,
    train_dataset=binary_classification_tokenized_train,
    eval_dataset=binary_classification_tokenized_eval,
    tokenizer=binary_classification_tokenizer,
    data_collator=binary_classification_data_collator,
    compute_metrics=compute_metrics
)

binary_classification_trainer.train()
binary_classification_results = binary_classification_trainer.evaluate()
print("Binary Classification Results:")
print(binary_classification_results)
binary_classification_preds = np.argmax(binary_classification_trainer.predict(binary_classification_tokenized_test).predictions, axis=-1)
print(classification_report(test_labels, binary_classification_preds, target_names=["Negative", "Positive"]))
end_binary_classification = time.time()
binary_classification_results["model"] = "binary-classification"
binary_classification_results["time_sec"] = end_binary_classification - start_binary_classification
results.append(binary_classification_results)

# Results Comparison (Benchmarking)

In [ ]:
df_results = pd.DataFrame(results)
print("\n Benchmark Summary:\n")
display(df_results)

## Confusion Matrix Visualization

In [ ]:
from sklearn.metrics import confusion_matrix

# Confusion Matrix visualization
def plot_conf_matrix(true, pred, model_name):
    cm = confusion_matrix(true, pred)
    plt.figure(figsize=(5,4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=["Negative", "Positive"], yticklabels=["Negative", "Positive"])
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title(f"Confusion Matrix - {model_name}")
    plt.show()

plot_conf_matrix(test_labels, nikolasmoya_preds, "nikolasmoya/imdb-binary-sentiment-analysis")
plot_conf_matrix(test_labels, roberta_preds, "kubi565/roberta-binary-sentiment-multilingual")
plot_conf_matrix(test_labels, binary_classification_preds, "autoevaluate/binary-classification")